### Notebook setup

In [0]:
!mv fast-bert_old/ fast-bert

In [0]:
# installing fast-bert from source
!git clone https://github.com/kaushaltrivedi/fast-bert.git/
%cd fast-bert
# !git config --global user.email "haard@gatech.edu"
# !git stash
!git reset --hard
!git checkout 10ee7a0d958d6c20c7af482a2d9a20db33a2b646
%cd ..

Cloning into 'fast-bert'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 821 (delta 74), reused 52 (delta 22), pack-reused 693
Receiving objects: 100% (821/821), 1.87 MiB | 3.52 MiB/s, done.
Resolving deltas: 100% (511/511), done.
/content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-model/fast-bert
HEAD is now at 77f09ad Merge pull request #204 from aaronbriel/abstract-hypers
error: Your local changes to the following files would be overwritten by checkout:
	container/bert/train
Please commit your changes or stash them before you switch branches.
Aborting
/content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-model


In [0]:
# %cd fast-bert/
!git checkout 10ee7a0d958d6c20c7af482a2d9a20db33a2b646

error: Your local changes to the following files would be overwritten by checkout:
	container/bert/train
Please commit your changes or stash them before you switch branches.
Aborting


In [0]:
!pip uninstall fast_bert
!pip list

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-model/'
# clone repo locally
!pip install --quiet transformers syntok 

%cd fast-bert
!pip install .
%cd ..

# need older versions of torch and torchvision to train with fp16 (might ask to restart runtime)
# !pip install https://download.pytorch.org/whl/cu100/torch-1.2.0-cp36-cp36m-manylinux1_x86_64.whl && pip install https://download.pytorch.org/whl/cu100/torchvision-0.4.0-cp36-cp36m-manylinux1_x86_64.whl
# install apex
# % cd apex
# ! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
# % cd ..

# Imports
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import torch
import re
import os
import sys
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))
from helpers.fast_bert_dataprep import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-model
     |████████████████████████████████| 573kB 32.8MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
     |████████████████████████████████| 1.0MB 52.7MB/s 
     |████████████████████████████████| 3.7MB 51.4MB/s 
/content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-model/fast-bert
Processing /content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-

In [0]:
!pip list | grep 'fast-bert'

fast-bert                1.7.0           /content/drive/My Drive/GaTech/Spring20/NL-CS7650/Controlled-textgen-model/fast-bert


In [0]:
"""
try:
    from apex import amp
except:
    % cd apex
    ! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
    % cd ..
"""

'\ntry:\n    from apex import amp\nexcept:\n    % cd apex\n    ! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./\n    % cd ..\n'

# FAST-BERT 

In [0]:
!nvidia-smi

Fri Apr 24 00:53:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!ps -aux|grep python

root          25  0.2  0.7 410964 102520 ?       Sl   00:18   0:04 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         124 15.6 36.1 28894324 4825844 ?    Ssl  00:38   2:14 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-cea6cacb-d6f2-40d6-922f-5646e9f091aa.json
root         584  0.0  0.0  39196  6604 ?        S    00:52   0:00 /bin/bash -c ps -aux|grep python
root         586  0.0  0.0  38572  4920 ?        S    00:52   0:00 grep python


In [0]:
!kill -9 124

In [0]:
from fast_bert.data_cls import BertDataBunch
from pathlib import Path

# EDIT 4/21 : training new model (512, bert-base-uncased)     - models2/
# EDIT 4/22 : Traning bert base uncased 5 more epochs - model2_2 /

MAX_LEN = 512 # previous model 300

text_col = 'script'
label_col = ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 
             'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
DATA_PATH = Path('./data/')
LABEL_PATH = DATA_PATH

train_file = 'fast_train_' + str(MAX_LEN) + '.csv'
val_file = 'fast_val_' + str(MAX_LEN) + '.csv'

print(os.path.exists(DATA_PATH/train_file))
print(os.path.exists(DATA_PATH/val_file))

# goodtogo = check_fastBert_data(MAX_LEN)
# if not goodtogo: die()

MODEL_NAME = 'bert-base-uncased'

databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer=MODEL_NAME,
                          train_file=train_file,
                          val_file=val_file,
                          label_file='fast_labels.csv',
                          text_col=text_col,
                          label_col=label_col,
                          batch_size_per_gpu=6,
                          max_seq_length=MAX_LEN,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='bert')

True
True


**NOTE** remember to change `usePretrained` to True if we've already have a fine-tuned model

In [0]:
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy, F1
import logging
from torch import Tensor

# EDIT 4/20 - my_accuracy_thresh changed thrshold to 0.5 and sigmoid=True
		# Also started using F1 score which uses 0.5 and sigmoid
		# Changed train with Fp16 option to False (apex isn't working again)

def my_accuracy_thresh(
    y_pred: Tensor,
    y_true: Tensor,
    thresh: float = 0.5,
    sigmoid: bool = True,
):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid:
        y_pred = y_pred.sigmoid()
    return ((y_pred > thresh) == y_true.bool()).float().mean().item()

logging.basicConfig(level=logging.NOTSET)
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy_thresh', 'function': my_accuracy_thresh}, {'name': 'F1', 'function':F1}]

OUTPUTDIR = Path('./models3/')

# MODEL_PATH = OUTPUTDIR/'model_epoch_0' # new with 512 max_len

usePretrained = True
if usePretrained:
	pretrained_path = Path('./models/model_out_0')
else:
	pretrained_path = MODEL_NAME

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path=pretrained_path,  #(to use saved model)
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUTDIR,
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=False,
						is_fp16=False,
						multi_label=True,
						logging_steps=50)
# set weight_decay
learner.weight_decay = 0.01

INFO:transformers.configuration_utils:loading configuration file models/model_out_0/config.json
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 11,
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "

In [0]:
learner.validate()

In [0]:
lst = list(learner.model.named_parameters())
print(lst[-1])
print(lst[-2])
# print(lst[-3])

In [0]:
!ls 

In [0]:
learner.fit(epochs=5,
			lr=6e-4,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")
# learner.save_model() # no need modified library file to save after every epoch

# simple inference right after training
# texts = ['I really love the Netflix original movies',
# 		 'this movie is not worth watching']
# predictions = learner.predict_batch(texts)

INFO:root:***** Running training *****
INFO:root:  Num examples = 45856
INFO:root:  Num Epochs = 5
INFO:root:  Total train batch size (w. parallel, distributed & accumulation) = 6
INFO:root:  Gradient Accumulation steps = 1
INFO:root:  Total optimization steps = 38215


*******OPTIMIZER CHANGED********
[{'params': [Parameter containing:
tensor([[-0.0306, -0.0029,  0.0228,  ...,  0.0155,  0.0173,  0.0010],
        [ 0.0131,  0.0061,  0.0118,  ..., -0.0387,  0.0122, -0.0007],
        [ 0.0045, -0.0320,  0.0147,  ..., -0.0204,  0.0235, -0.0244],
        ...,
        [ 0.0121,  0.0125,  0.0229,  ..., -0.0222,  0.0198,  0.0027],
        [ 0.0374, -0.0017,  0.0237,  ...,  0.0140, -0.0067, -0.0313],
        [ 0.0157,  0.0095,  0.0401,  ..., -0.0061,  0.0074,  0.0092]],
       device='cuda:0', requires_grad=True)], 'weight_decay': 0.01}, {'params': [Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)], 'weight_decay': 0.0}]


INFO:root:Running evaluation
INFO:root:  Num examples = 5282
INFO:root:  Batch size = 12


ValueError: ignored

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install --quiet gputil psutil humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 9.9 GB  |     Proc size: 4.9 GB
GPU RAM Free: 357MB | Used: 15923MB | Util  98% | Total     16280MB


# PREDICTION

In [0]:
%load_ext tensorboard
%tensorboard --logdir models/tensorboard/

In [0]:
df = pd.read_csv("data/output.csv")
df

,Genre,Seed_Text,Generations_GPT2,BART_gen,2gram_gen,3gram_gen
0,<Action>,The man went to the park,.\n\n PIKE\n How'd you get out here?\n\n FATHE...,The man went to the park. The man then went to...,you took from the sofa staring at the plug...t...,i n - vehicles pulling in front of their elabo...
1,<Comedy>,The man went to the park,.\n\n PAUL\n Turn around.\n\n FATHER\n Stop it...,The man went to the park. The man then went to...,": JENNIFER I'm sorry . There's the whole body,...",on just one afternoon and if they only had eye...
2,<Thriller>,The man went to the park,.\n\n EXT. park--\n\n PINGER'S POLICE CAR--LAT...,The man went to the park. The man then went to...,blocks from him immediately . When one of bloo...,i n g beneath as he's been crudely hacked out ...
3,<Horror>,The man went to the park,", and I\n got off.\n \n Reilly walks past Holl...",The man went to the park. The man then went to...,n roll that . SMALL APARTMENT BUILDING - NIGHT...,on just a WAITRESS and some of the trees! SCOT...
4,<Romance>,The man went to the park,.\n\n Ray waves dismissively.\n\n RAY\n Then I...,The man went to the park. The man then went to...,now . JAVED O/S) No . But it was looking aroun...,", then hesitates to open death's icy hand can ..."
...,...,...,...,...,...,...
135,<Thriller>,Frank feels their eyes on him as he begins to ...,\n There is a brief silence as he moves to hi...,Frank feels their eyes on him as he begins to ...,GABRIELLA They rush into an instant . A BLUE S...,"what kind of history? JACK Of course, another ..."
136,<Horror>,Frank feels their eyes on him as he begins to ...,\n \n FURTHER ON\n \n as he looks at the watch...,Frank feels their eyes on him as he begins to ...,) Dr. Linsky here . GRENDEL Rizzo pull it on a...,the sand-cat . EXT . GASPARILLA STAY HERE AND ...
137,<Romance>,Frank feels their eyes on him as he begins to ...,\n Dallas Buyers Club / Green Revision / Dec 2...,Frank feels their eyes on him as he begins to ...,facetious) Wait Steven has his eyes and the ti...,the glove . They start up the phone . He usual...
138,<Sci-Fi>,Frank feels their eyes on him as he begins to ...,\n Dallas Buyers Club / Green Revision / Dec 2...,Frank feels their eyes on him as he begins to ...,station . Up a yellowish bright orange light S...,"? JOHNNY Actually, he wears a small armory, Tr..."


In [0]:
gpt_gen = df.iloc[:, 2].values.tolist()
bart_gen = df.iloc[:, 3].values.tolist()
bi_gen = df.iloc[:, 4].values.tolist()
tri_gen = df.iloc[:, 5].values.tolist()

In [0]:
# Inference using saved fine-tuned model
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = "models/model_out_4/"
LABEL_PATH = "data/"

predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=True,
				model_type='bert',
				do_lower_case=False)

gpt_pred = predictor.predict_batch(gpt_gen)
bard_pred = predictor.predict_batch(bart_gen)
bi_pred = predictor.predict_batch(bi_gen)
tri_pred = predictor.predict_batch(tri_gen)

# # Single prediction
# single_prediction = predictor.predict("just get me result for this text")

# # Batch predictions
# texts = [
# 	"this is the first text",
# 	"this is the second text"
# 	]

# multiple_predictions = predictor.predict_batch(texts)


In [0]:
print(bi_pred)

[[('Action', 0.9512056112289429), ('Adventure', 0.9470341801643372), ('Fantasy', 0.45823681354522705), ('Comedy', 0.3223428428173065), ('Sci-Fi', 0.31907835602760315), ('Thriller', 0.1256701648235321), ('Crime', 0.052951350808143616), ('Horror', 0.044540174305438995), ('Drama', 0.04276547580957413), ('Romance', 0.04145081713795662), ('Mystery', 0.032077912241220474)], [('Comedy', 0.9238592982292175), ('Drama', 0.2497091144323349), ('Adventure', 0.16362060606479645), ('Romance', 0.1495499163866043), ('Fantasy', 0.11264465749263763), ('Action', 0.06058504432439804), ('Sci-Fi', 0.02808377332985401), ('Horror', 0.018938448280096054), ('Crime', 0.018001550808548927), ('Mystery', 0.007300133351236582), ('Thriller', 0.005806987173855305)], [('Drama', 0.9575962424278259), ('Thriller', 0.4938738942146301), ('Crime', 0.3441929221153259), ('Action', 0.23025652766227722), ('Mystery', 0.2118643969297409), ('Romance', 0.033690325915813446), ('Sci-Fi', 0.027476079761981964), ('Horror', 0.027355153113

In [0]:
total = len(gpt_gen)
scores = {
    "gpt": [0, 0],
    "bart" : [0, 0],
    "bi" : [0, 0],
    "tri" : [0, 0]
          }
for i, g in enumerate(df['Genre']):
    gg = g.replace("<", "").replace(">", "")
    three_gpt = [a[0] for a in gpt_pred[i][:3]]
    # print(gg, three_gpt)
    # print(gg in three_gpt)
    # die()
    if gg == three_gpt[0]:
        scores['gpt'][0] += 1
        scores['gpt'][1] += 1
    elif gg in three_gpt:
        scores['gpt'][1] += 1

    three_bart = [a[0] for a in bard_pred[i][:3]]
    if gg == three_bart[0]:
        scores['bart'][0] += 1
        scores['bart'][1] += 1
    elif gg in three_bart:
        scores['bart'][1] += 1

    three_bi = [a[0] for a in bi_pred[i][:3]]
    # print(gg, three_bi)
    if gg == three_bi[0]:
        scores['bi'][0] += 1
        scores['bi'][1] += 1
    elif gg in three_bi:
        scores['bi'][1] += 1

    three_tri = [a[0] for a in tri_pred[i][:3]]
    if gg == three_tri[0]:
        scores['tri'][0] += 1
        scores['tri'][1] += 1
    elif gg in three_tri:
        scores['tri'][1] += 1

scores

{'bart': [4, 32], 'bi': [33, 105], 'gpt': [19, 53], 'tri': [44, 102]}

In [0]:
for mod in scores:
    scores[mod][0] /= total
    scores[mod][1] /= total

scores

{'bart': [0.02857142857142857, 0.22857142857142856],
 'bi': [0.2357142857142857, 0.75],
 'gpt': [0.1357142857142857, 0.37857142857142856],
 'tri': [0.3142857142857143, 0.7285714285714285]}

In [0]:
# genre, seed_text, gpt_pred, bi_pred, tri_pred
gpt_pd = pd.Series(["<"+g[0][0]+">" for g in gpt_pred])
bart_pd = pd.Series('<'+b[])
bi_pd = pd.Series(['<'+b[0][0]+'>' for b in bi_pred])
tri_pd = pd.Series('<'+t[0][0]+'>' for t in tri_pred)


# df = pd.DataFrame(columns = ['Column_name#1', 'Column_name#1'])

result_df = pd.DataFrame(columns=['Genre', 'Seed_Text', 'BERT_cls_gpt_gen', 'BERT_cls_2gram', 'BERT_cls_3gram'])
result_df['Genre'] = df['Genre']
result_df['Seed_Text'] = df['Seed_Text']
result_df['BERT_cls_gpt_gen'] = gpt_pd
result_df['BERT_cls_2gram'] = bi_pd
result_df['BERT_cls_3gram'] = tri_pd

In [0]:
result_df.to_csv('data/BERT_eval.csv', index=False)

# Prepare scripts

1. Read `data/train.csv`
2. Remove rare genres
3. Create Genre-to-int and int-to-genre mappings
4. Split into Train and Val sets

*Next* use the train and val data to train sequence classifier

In [0]:
def getTrainValSplit(min_freq=50, split_train=0.9, data_file='data/train.csv', verbose=True):
    """
    create train-val split from data from `data_file`
    """
    df = pd.read_csv(data_file, delimiter=',')
    if verbose: print('all columns:', df.columns)
    # drop cols < freq
    stats = df.iloc[:, 4:].sum(axis=0) 
    cols = list(stats[stats < min_freq].index)
    for col in cols:
        df.drop(col, axis=1, inplace=True)
    if verbose: print("new genre totals:\n", df.iloc[:, 4:].sum())
    # drop rows that don't belong to any genres now
    onehotlabels = df.iloc[:, 4:]
    rows = onehotlabels.sum(axis=1)
    if verbose: print('total rows before: ', len(rows))
    # remove rows with sum 0 ; no labels
    rows = list(rows[rows==0].index)
    for row in rows:
        df.drop(row, axis=0, inplace=True)
    if verbose: print('total rows after: ', len(df))

    msk = np.random.rand(len(df)) < split_train
    train = df[msk]
    val = df[~msk]
    if verbose: print('train:', len(train), '; val:', len(val), '; all:', len(df))
    split_stats = pd.concat([train.iloc[:, 4:].sum(), val.iloc[:, 4:].sum(), train.iloc[:, 4:].sum()/df.iloc[:, 4:].sum()], axis=1)
    if verbose: print(split_stats)

    # label->int and int->label mapping
    if verbose: print('creating label str<->int mappings')
    cols = df.columns
    idx = 0
    genre_to_int = {}
    int_to_genre = {}
    for c in cols[4:]:
        int_to_genre[idx] = c
        genre_to_int[c] = idx
        idx += 1
    if verbose: print(genre_to_int)
    if verbose: print(int_to_genre)

    return train, val, genre_to_int, int_to_genre

In [0]:
genre_movie_count = {}

def prep_data(dframe, tokenizer, max_len=300):
    input_ids = []
    labels = []
    attention_masks = []

    shortSkipped = 0

    for i in range(len(dframe)):
        thisrow = dframe.iloc[i, :]

        # check label (if multiple labels, skip)
        onehot = thisrow[4:]
        print(onehot.values)
        die()
        if onehot.sum() == 1:
            thislabel = onehot[onehot==1]
            thislabel = list(thislabel.index)[0]
            thislabel = genre_to_int[thislabel]
            # print(thislabel)
        else: 
            many = list(onehot[onehot==1].index)
            many = '.'.join(many)
            thislabel = len(genre_to_int)
            genre_to_int[many] = thislabel
            int_to_genre[thislabel] = many
        # count
        if thislabel in genre_movie_count:
            genre_movie_count[thislabel] += 1
        else:
            genre_movie_count[thislabel] = 1
        continue

        # encode script
        script = re.sub('\s+',' ', str(thisrow.script))
        # print('script len', len(script))
        # print('split len', len(script.split()))
        if len(script) < max_len:
            # print('skipping movie:', thisrow.title, 'because short script')
            shortSkipped += 1
            continue
        encoded = tokenizer.encode(script, add_special_tokens=False)
        # print('encoded len', len(encoded))
        # print(len(script), len(encoded), type(encoded))
        chunked = list(chunkset(encoded, max_len-2)) 
        # script_chunks = list(chunkset(script, max_len))
        # print(torch.tensor(chunked[0]))
        # print('total chunks', len(chunked))
        for chunk in chunked:
            encoded_dict = tokenizer.encode_plus(
                            chunk,             # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_len,    # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True, # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                            # truncation_strategy = 'do_not_truncate'
                    )
            input_ids.append(encoded_dict['input_ids'])
            # print(ids.shape)
            # print(ids)
            attention_masks.append(encoded_dict['attention_mask'])
            # print(msks.shape)
            # print(msks)
            labels.append(thislabel)
    for genre in genre_movie_count:
        print(int_to_genre[genre], genre_movie_count[genre])    
    
    return None, None, None
    print('movies skipped', shortSkipped + multiGenreSkipped)
    print('short scripts', shortSkipped)
    print('scripts with multiple genres', multiGenreSkipped)
    return input_ids, attention_masks, labels

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

train_ids, train_attention_masks, train_labels = prep_data(df, tokenizer, max_len=64)

Loading BERT tokenizer...
[0 0 1 0 0 0 0 0 1 0 0]


NameError: ignored

# Prep data for fast-BERT

Reduce classification problem (to be used with fast-BERT)

> Create new chunked data (as specified by `max_len`)

> Convert save chunked data into `data/fast_train.csv` and 
`data/fast_val.csv`

**NOTE**: might have to do the same for test data (held out)

**Shuffle?** data is in alphabetical order (hopefully fast-bert takes care of 
choosing batches randomly instead of just indexing through csv)

In [0]:
# for fast_labels.csv
label_col = []
with open('data/fast_labels.csv', 'w') as f:
    for i in range(len(int_to_genre)):
        label_col.append(int_to_genre[i])
        f.write(int_to_genre[i] + '\n')
print(label_col)

['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']


In [0]:
# 
def chunkDataframe(dframe, max_len=300):
    new_frame = pd.DataFrame().reindex_like(train)
    new_frame.drop(new_frame.index, axis=0, inplace=True)
    for i in range(0, len(dframe)):
        if i % 20 == 0: print('processing', i, 'of', len(dframe),' -- title:', dframe.iloc[i].title)
        thisrow = dframe.iloc[i]
        chunked_script = chunkScript(thisrow.script, max_len=max_len)
        thisrow.at['script'] = ''
        for c in chunked_script:
            thisrow.at['script'] = c
            new_frame = new_frame.append(thisrow, ignore_index=True)
    return new_frame

In [0]:
# FINAL STEP (execute only once / or to generate of with different max_len)
new_val = chunkDataframe(val, max_len=512)
new_val.to_csv('data/fast_val_512.csv')
print('chunked val:', len(new_val), '; original_val', len(val))

new_train = chunkDataframe(train, max_len=512)
new_train.to_csv('data/fast_train_512.csv')
print('chunked train (smaller rows):', len(new_train), '; original_train:', len(train))

print("---------- saved train and test CSV ------------")

In [0]:
# # fast_train.csv
# new_train.to_csv('data/fast_train.csv')
# # fast_val.csv
# new_val.to_csv('data/fast_val.csv')
# text_col = 'script'

# Data preprocessing

### Syntok (tokenwise preprocessing)

https://github.com/fnl/syntok

To insert [SEP] tokens at the end of sentences in movie scripts

In [0]:
!pip install syntok

  Created wheel for syntok: filename=syntok-1.2.2-cp36-none-any.whl size=20724 sha256=a34abe1879706677a7a1b2822b418540d273a9e368d484c633dcae243068ecec
  Stored in directory: /root/.cache/pip/wheels/b4/b0/d2/ffdbbc1a16cb37e580fb7b3a6fbaaf09c7f7c163981db385b3
Successfully built syntok


In [0]:
import syntok.segmenter as segmenter

In [0]:
def chunkset(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

def __extractValueFromToken__(tokenLst):
    return [t.value for t in tokenLst]

def chunkScript(movie, max_len=300):
    """
    add `[SEP]` token at the end of each sentence in script
    """
    outscript = ['']
    # movie = re.sub('\s+',' ', movie)
    prev_len = 0
    try:
        segmentation = list(segmenter.process(movie))

        for paragraph in segmentation:
            for s in paragraph:
                sent = ' '.join([t.value for t in s]) + ' [SEP]'
                if (prev_len + len(s) < max_len-2):
                    outscript[-1] += sent
                    prev_len += len(s)
                elif (len(s) < max_len-2):
                    outscript.append(sent)
                    prev_len = len(s)
                else: 
                    # new sentence is longer than max_len
                    # print(len(outscript))
                    outscript += [' '.join(__extractValueFromToken__(s2)) for s2 in chunkset(s, max_len - 1)]
                    # print(len(outscript))
                    prev_len = len(outscript[-1])
    except:
        print("segmentatin error for movie:", movie)
        return []
            
    return outscript
# out = chunkScript(train.iloc[0].script, max_len=128)
# len(out)
# print(out[-1])

## Data Prep

# New Section